In [1]:
#구글 드라이브와 연결하기
from google.colab import drive
drive.mount('./gdrive',force_remount=True)

Mounted at ./gdrive


In [2]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [2]:
#말뭉치 내려받기 및 전처리
from Korpora import Korpora
nsmc = Korpora.load('nsmc',force_download=True) #네이버 영화 리뷰 NSMC


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 88.6MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.5MB/s]                            


In [5]:
import os
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')
write_lines('/content/train.txt', nsmc.train.get_all_texts())
write_lines('/content/test.txt', nsmc.test.get_all_texts())

In [10]:
#gpt 계열에선 bpe를 토크나이저 기법으로 사용한다. 유니코드 바이트 기준 bpe를 사용하면 토큰 문제에서 비교적 자유롭다.
os.makedirs('/gdrive/My Drive/nlpbook/bbpe',exist_ok=True) #바이트 수준 bpe어휘집합 구축 결과 저장 디렉터리

In [11]:
from tokenizers import ByteLevelBPETokenizer #어휘 집합 구축 대상 말뭉치를 유니코드 바이트 수준으로 만들어줌
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files = ['/content/train.txt','/content/test.txt'], #학습 말뭉치를 리스트 형태로
    vocab_size = 10000, #어휘 집합 크기 조절
    special_tokens = ["[PAD]"] #특수 토큰 추가
)
bytebpe_tokenizer.save_model('/gdrive/My Drive/nlpbook/bbpe')

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [7]:
#학습된 모델 불러와서 테스트
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files = ['/content/train.txt','/content/test.txt'],
    vocab_size=10000
)
os.makedirs('/gdrive/My Drive/nlpbook/wordpiece',exist_ok=True)
wordpiece_tokenizer.save_model('/gdrive/My Drive/nlpbook/wordpiece')

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [12]:
#gpt와 bert모델에서 사용되는 어휘 집합을 파일로 만들어놔싿.
#이번에는 문장을 토큰화하고 해당 토큰들을 모델의 입력으로 만드는 과정을 실습한다.

#gpt 입력값 만들기
#gpt의 입력값을 만들려면 토크나이저부터 준비해야된다.
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe") #전에 만들어놨던 네이버 영화리뷰에 대한 사전학습 토크나이저
tokenizer_gpt.pad_token = "[PAD]"

In [13]:
sentences = ['아 더빙.. 진짜 짜증나네요 목소리',
             '흠...포스터보고 초딩영화인줄.... 오버연기조차 가볍지 않구나',
             '별루 였다..']
tokenized_sentence = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]


In [14]:
print(tokenized_sentence) #토큰화가 잘 된걸 확인할 수 있다.
#이 코드는 맛보기이고 진짜로 하려면 다음과 같이 하면된다.

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'], ['íĿł', '...', 'íı¬ìĬ¤íĦ°', 'ë³´ê³ł', 'Ġì´ĪëĶ©', 'ìĺģíĻĶ', 'ìĿ¸ì¤Ħ', '....', 'Ġìĺ¤ë²Ħ', 'ìĹ°ê¸°', 'ì¡°ì°¨', 'Ġê°Ģë³į', 'ì§Ģ', 'ĠìķĬ', 'êµ¬ëĤĺ'], ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]


In [15]:
batch_inputs = tokenizer_gpt(
    sentences,padding = 'max_length', #문장의 최대 길이에 맞춰서 패딩
    max_length = 12, #문장의 토큰 기준 최대 길이
    truncation = True #문장 잘림 허용 옵션
)


In [17]:
print(batch_inputs) #토큰화가 잘되어있는 것을 확ㅇ니할 수 있다.
#각 토큰을 인덱스로 바꾼 것을 input_ids에서 보여준다.
print(batch_inputs['attention_mask'])
#attention mask는 일반 토큰이 자리한 곳 (1) 과 패딩 토큰이 자리한 곳 (0)을 나타낸다.


{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0], [3693, 336, 2876, 758, 2883, 356, 2118, 422, 4444, 875, 2960, 7292], [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}
[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]


In [18]:
#이번엔 bert모델의 입력값을 만들어보자
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    '/gdrive/My Drive/nlpbook/wordpiece',
    do_lower_case = False
)
bert_tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [19]:
print(bert_tokenized_sentences)
#이것 또한 맛보기이고 진짜 입력을 만들고 싶으면 다음과 같이 한다.

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'], ['흠', '.', '.', '.', '포스터', '##보고', '초딩', '##영화', '##인줄', '.', '.', '.', '.', '오버', '##연기', '##조차', '가볍', '##지', '않', '##구나'], ['별루', '였다', '.', '.']]


In [20]:
bert_batch_inputs = tokenizer_bert(
    sentences,
    padding = 'max_length',
    max_length = 12,
    truncation = True
)


In [21]:
print(bert_batch_inputs)
#input_ids는 토큰화 된 인덱스를 보여주고 각각 2로 시작해서 3으로 끝나는데, 2는 CLS, 3은 SEP이다.
#또한 token_type_ids라는 입력값도 만들어지는데 이는 세그먼트에 해당되는 것이고
#bert모델은 기본적으로 문장 2개를 입력받으며 첫번째는 0, 두번째는 1로 표기해주는 것이다.

{'input_ids': [[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0], [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 2697, 16, 3], [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}


In [25]:
print('learning trainsformers!!')

learning trainsformers!!


In [26]:
#사전학습된 bert 모델을 가지고 문장을 벡터로 변환하기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case = False
)


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [27]:
from transformers import BertConfig, BertModel
#다음 단계에서 모델을 초기화해준다. 중요한 것은 사용 대상 Bert모델이 pretrained할 때 썼던 토크나이저를 그대로 사용해서 벡터 변환에 문제가 없다는 것이다.
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config

)
#pretrained_model_config는 bert모델을 pretrain할 때 설정했던 내용이 담겨있다.
#레이어, 헤드, 어휘집합 등 확인할 수 있다.


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
print(pretrained_model_config)

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}



In [29]:
#입력값 만들기
sentences = ['안녕하세요','박현석입니다.']
feats = tokenizer(
    sentences,
    max_length = 10,
    padding = 'max_length',
    truncation = True
)

In [30]:
print(feats)

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 1481, 4175, 4731, 8074, 17, 3, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}


In [31]:
#벡터 구하기
import torch
feats = {k:torch.tensor(v) for k, v in feats.items()}

In [33]:
outputs = model(**feats) #여러 출력 결과를 한데 묶은 것이다.
print(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[-1.4258,  1.0043,  1.0540,  ..., -0.8624,  0.3832, -0.2871],
         [-1.4290, -0.2269, -0.1123,  ...,  0.1178, -0.3739,  0.9396],
         [-0.3146,  0.4470,  0.9300,  ..., -0.0909, -0.0030, -0.2924],
         ...,
         [-0.8917,  0.6938,  1.3166,  ...,  0.4647, -0.0512, -0.4810],
         [-1.2073,  0.6176,  1.2419,  ...,  0.2896,  0.3558, -0.4357],
         [-1.8145,  0.2750,  0.7450,  ...,  0.1547, -0.7292, -0.0665]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [34]:
print(outputs.last_hidden_state.shape) #2개 문장 max length 10 768차원 벡터로 변환됨을 확인

torch.Size([2, 10, 768])
